In [12]:
import json
materials = []
with open('/Users/yao/Desktop/band offset/gap3.csv', 'r') as f:
    for line in f:
        mid = line.split(',')[2].rstrip()
        if mid != 'NA':
            materials.append(mid)

In [13]:
from pymatgen.matproj.rest import MPRester
import os
from pymatgen.core.periodic_table import Element
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.sets import MPStaticSet

os.makedirs('DOS-2',exist_ok=True)
with MPRester(os.environ['MAPI_KEY']) as m:
    for mid in materials:
        try:
            formula = m.get_data(mid, data_type="vasp", prop="pretty_formula")[0]['pretty_formula']
            structure = m.get_structure_by_material_id(mid)
            usetting = {'EDIFF': 1e-5}
            sset = MPStaticSet(structure, reciprocal_density=100, user_incar_settings=usetting)
            sset.write_input('DOS-2/'+formula)
        except:
            print(mid)
        

In [2]:
from pymatgen.matproj.rest import MPRester
import os
from pymatgen.io.vasp.sets import MPHSERelaxSet
os.makedirs('DOS-HSE',exist_ok=True)
with MPRester(os.environ['MAPI_KEY']) as m:
    for mid in materials:
        formula = m.get_data(mid, data_type="vasp", prop="pretty_formula")[0]['pretty_formula']
        structure = m.get_structure_by_material_id(mid)
        usetting = {'EDIFF': 1e-5, 'ISMEAR': -5, 'IBRION': -1, 'NSW': 0,
                   'AMIX': 0.1, 'TIME': 0.4, 'NKRED': 1, 'LASPH': '.TRUE.', 'LMAXMIX': 4}
        sset = MPHSERelaxSet(structure, user_incar_settings=usetting)
        sset.write_input('DOS-HSE/'+formula)

In [21]:
## k-test
materials = ['mp-2534', 'mp-2133']

from pymatgen.matproj.rest import MPRester
import os
from pymatgen.core.periodic_table import Element
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.sets import MPStaticSet

os.makedirs('DOS-ktest',exist_ok=True)
with MPRester(os.environ['MAPI_KEY']) as m:
    for mid in materials:
        formula = m.get_data(mid, data_type="vasp", prop="pretty_formula")[0]['pretty_formula']
        structure = m.get_structure_by_material_id(mid)
        usetting = {'EDIFF': 1e-5}
        for density in [10, 30, 100, 300, 1000]:
            sset = MPStaticSet(structure, reciprocal_density=density, 
                               user_incar_settings=usetting, force_gamma = False)
            sset.write_input('DOS-ktest/'+formula+'-k'+str(density))


In [1]:
## BS calculation
materials = ['mp-2534'] #, 'mp-2133']

from pymatgen.matproj.rest import MPRester
import os
from pymatgen.core.periodic_table import Element
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.sets import MPStaticSet, MPNonSCFSet

os.makedirs('DOS-BS',exist_ok=True)
with MPRester(os.environ['MAPI_KEY']) as m:
    for mid in materials:
        formula = m.get_data(mid, data_type="vasp", prop="pretty_formula")[0]['pretty_formula']
        #structure = m.get_structure_by_material_id(mid)
        #usetting = {'EDIFF': 1e-5}
        prev_dir = '/Users/yao/Google Drive/data/semiconductor/DOS-PBE/'+formula
        sset = MPNonSCFSet.from_prev_calc(prev_dir)                         
        sset.write_input('DOS-BS/'+formula)


In [9]:
import glob

dir_list = glob.glob('/Users/yao/Google Drive/data/semiconductor/DOS-PBE/*')
for d in dir_list:
    formula = d.split('/')[7]



'ZnTe'